## 환경 설정

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("ChatOllama")

## Ollama 모델 사용

한국어 잘하는 Open 모델 

**참고**

각 모델의 라이센스를 반드시 확인 후 사용해주세요.

- EXAONE-3.5 모델(gguf): https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct-GGUF
- gemma2-27b: https://ollama.com/library/gemma2:27b
- EEVE-Korean-10.8B(gguf): https://huggingface.co/teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf
- Qwen2.5-7B-Instruct-kowiki-qa-context(gguf): https://huggingface.co/teddylee777/Qwen2.5-7B-Instruct-kowiki-qa-gguf

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_teddynote.messages import stream_response

# Ollama 모델 지정
llm = ChatOllama(
    model="exaone",
    temperature=0,
)

# 프롬프트 정의
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# 체인 생성
chain = prompt | llm | StrOutputParser()

# 스트림 출력
answer = chain.stream({"topic": "deep learning"})
stream_response(answer)

In [ ]:
# gemma2-27b
llm = ChatOllama(
    model="gemma2:27b",
    temperature=0,
)

# 주제를 기반으로 짧은 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(
    "Answer the following question in Korean.\n\nQuestion: {question}\n\nAnswer:"
)

# LangChain 표현식 언어 체인 구문을 사용합니다.
chain = prompt | llm | StrOutputParser()

# 체인 실행
answer = chain.stream({"question": "python 코드로 피보나치 수열을 구현해보세요."})
stream_response(answer)

## OllamaEmbeddings 사용

링크: https://ollama.com/library/bge-m3

명령어

`ollama pull bge-m3`

In [5]:
from langchain_ollama import OllamaEmbeddings

# 임베딩 설정
embeddings = OllamaEmbeddings(
    model="bge-m3",
)

### 코사인 유사도 계산

In [6]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

유사도 계산을 위한 임베딩을 수행합니다.

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings.embed_documents(sentences)

In [8]:
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

유사도 계산 결과는 다음과 같습니다.

In [ ]:
# sentence1 = "안녕하세요? 반갑습니다."
# sentence2 = "안녕하세요? 반갑습니다!"
# sentence3 = "안녕하세요? 만나서 반가워요."
# sentence4 = "Hi, nice to meet you."
# sentence5 = "I like to eat apples."

for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

**실습에 활용한 문서**

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


In [10]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_teddynote.messages import stream_response

# 문서 로드
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_docs = loader.load_and_split(text_splitter)

# 임베딩 설정
embeddings = OllamaEmbeddings(
    model="bge-m3",
)

# 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings)

# 검색기 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 프롬프트 로드
prompt = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

Please follow these instructions:

1. Analyze the content of the source documents: 
2. The name of each source document is at the start of the document, with the <document> tag.

-----

Output format should be like this:

(Your comprehensive answer to the question)

**Source**
- [1] Document source with page number
- [2] Document source with page number
(...)

-----

### Here is the context that you can use to answer the question:

#Context: 
{context}

### Here is user's question:

{question}

Your answer to the question:

### Answer:"""
)


def format_docs(docs):
    return "\n\n".join(
        f"<document><content>{doc.page_content}</content><page>{doc.metadata['page']}</page><source>{doc.metadata['source']}</source></document>"
        for doc in docs
    )


# Ollama 모델 지정
llm = ChatOllama(
    model="exaone",
    temperature=0,
)

# 체인 생성
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "삼성전자가 개발한 생성형 AI 의 이름은?"

# 체인 실행
response = chain.stream(question)
# 스트림 출력
stream_response(response)